In [168]:
import pandas as pd
import numpy as np

In [169]:
exp_num = 2
unique_huge = pd.read_csv("../hugecookies/hugecookies-{}.csv".format(exp_num))

In [170]:
unique_huge.shape

(197370, 6)

In [171]:
unique_huge.dropna(inplace=True)
unique_huge.shape

(191841, 6)

In [172]:
unique_huge = unique_huge[unique_huge['value'].str.len() >= 8]
unique_huge.shape

(146194, 6)

In [173]:
unique_huge.drop(unique_huge[unique_huge.name.str.contains('Test|test|lang|SameSite|Samesite|sameSite|samesite|expires|Path|path|domain|Expires|max-age')].index, inplace=True)
unique_huge.drop(unique_huge[unique_huge.value.isin(['1','0','2','true','yes','YES','ok','nan','none','None','NO_DATA','null','setstatuscode~~1'])].index, inplace=True)
unique_huge.shape

(141778, 6)

In [174]:
# unique_huge.value.unique().tolist()

In [175]:
unique_huge['namevalue'] = list(map(lambda a, b: (a, b), unique_huge["name"].astype(str), unique_huge["value"].astype(str)))

In [176]:
byoccur = unique_huge.groupby(["namevalue", "context_id"], as_index = False)['time_stamp'].count()
byoccur
#temp2.to_csv("unique_namevalue-3.csv", index=False)

,namevalue,context_id,time_stamp
0,"(113, 20200501)",1,1
1,"(131, 20200501)",1,1
2,"(131, 20200501)",2,1
3,"(33x_ps, u%3D76839448493146%3As1%3D15883690020...",2,20
4,"(A3, d=AQABBFCWrF4CEOk_HrROnnxMZ2lVKqQaT5AFEgE...",2,3
...,...,...,...
1693,"(zuid, KHZ89MT9KlK-f1O8NSb0)",1,18
1694,"(zuid, KHZ89MT9KlK-f1O8NSb0)",2,10
1695,"(zuuid, da2a4c3b-a7c3-4004-9688-93ae99812a48)",2,4
1696,"(zuuid_lu, 1588368995)",2,2


In [177]:
# clean host domain 
temp = list(map(lambda x:".".join(x.split(".")[:2]).replace('https://','').replace('www',''), unique_huge['host'].tolist()))                    
len(temp)
unique_huge['host_clean'] = temp


In [178]:
# byhost = pd.DataFrame(unique_huge.groupby(["namevalue", "context_id"], as_index = False)['host_clean'].apply(list))
byhost = pd.DataFrame(unique_huge.groupby(["namevalue", "context_id"], as_index = False)['host_clean'].apply(set).reset_index(name='byhost'))
byhost

,namevalue,context_id,byhost
0,"(113, 20200501)",1,{.deepintent}
1,"(131, 20200501)",1,{match.deepintent}
2,"(131, 20200501)",2,{.deepintent}
3,"(33x_ps, u%3D76839448493146%3As1%3D15883690020...",2,"{cms-xch-chicago.33across, .33across, ssc.33ac..."
4,"(A3, d=AQABBFCWrF4CEOk_HrROnnxMZ2lVKqQaT5AFEgE...",2,"{ups.analytics, pr-bh.ybp, .yahoo}"
...,...,...,...
1693,"(zuid, KHZ89MT9KlK-f1O8NSb0)",1,"{b1sync.zemanta, .zemanta}"
1694,"(zuid, KHZ89MT9KlK-f1O8NSb0)",2,"{b1sync.zemanta, .zemanta}"
1695,"(zuuid, da2a4c3b-a7c3-4004-9688-93ae99812a48)",2,"{a.sportradarserving, .sportradarserving}"
1696,"(zuuid_lu, 1588368995)",2,"{a.sportradarserving, .sportradarserving}"


In [179]:
highlight = pd.merge(byoccur, byhost, on=['namevalue',  'context_id'])
highlight

,namevalue,context_id,time_stamp,byhost
0,"(113, 20200501)",1,1,{.deepintent}
1,"(131, 20200501)",1,1,{match.deepintent}
2,"(131, 20200501)",2,1,{.deepintent}
3,"(33x_ps, u%3D76839448493146%3As1%3D15883690020...",2,20,"{cms-xch-chicago.33across, .33across, ssc.33ac..."
4,"(A3, d=AQABBFCWrF4CEOk_HrROnnxMZ2lVKqQaT5AFEgE...",2,3,"{ups.analytics, pr-bh.ybp, .yahoo}"
...,...,...,...,...
1693,"(zuid, KHZ89MT9KlK-f1O8NSb0)",1,18,"{b1sync.zemanta, .zemanta}"
1694,"(zuid, KHZ89MT9KlK-f1O8NSb0)",2,10,"{b1sync.zemanta, .zemanta}"
1695,"(zuuid, da2a4c3b-a7c3-4004-9688-93ae99812a48)",2,4,"{a.sportradarserving, .sportradarserving}"
1696,"(zuuid_lu, 1588368995)",2,2,"{a.sportradarserving, .sportradarserving}"


In [180]:
# select only across contexts
ltd = []
Counter = collections.Counter(highlight.namevalue.tolist())

In [181]:
for key, value in Counter.items():
    if value == 1:
        ltd.append(key)

In [182]:
indtd = highlight[highlight['namevalue'].isin(ltd)].index
highlight.drop(indtd, inplace=True)

In [185]:
# highlight.namevalue.unique().tolist()

In [184]:
highlight.to_csv("../overlap-cookies/unique_namevalue-{}.csv".format(exp_num), index=False)